In [16]:
import json
import time
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

In [12]:
from geopy.geocoders import Nominatim
geo_local = Nominatim(user_agent='South Korea')

import requests
# 위도, 경도 반환 함수
def geocoding(address):
    try:
        url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
        headers = {"Authorization": "KakaoAK b661426628e6bb134e2766ad9592dd33"}
        api_json = json.loads(str(requests.get(url,headers=headers).text))
        address = api_json['documents'][0]['address']
        x_y = [str(address['y']), str(address['x'])]
        return x_y
    except:
        x_y = ['', '']
        return x_y
    
# 네이버 지도 검색 페이지 열기
driver.get("https://map.naver.com/v5/search")
key_word = '경주 십원빵'  # 검색어

# css 찾을때 까지 10초대기
def time_wait(num, code):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait

# frame 변경 메소드
def switch_frame(frame):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경

# 페이지 다운
def page_down(num):
    body = driver.find_element(By.CSS_SELECTOR, 'body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)

# css를 찾을때 까지 10초 대기
time_wait(10, 'div.input_box > input.input_search')

# 검색창 찾기
search = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
search.send_keys(key_word)  
search.send_keys(Keys.ENTER) 

sleep(5)

# frame 변경
time_wait(10, 'iframe#searchIframe')
switch_frame('searchIframe')

sleep(5)

page_down(40)
sleep(2)

# 가게 리스트
store_list = driver.find_elements(By.CSS_SELECTOR, 'li.VLTHu')
# 페이지 리스트
next_btn = driver.find_elements(By.CSS_SELECTOR, '.zRM9F> a')

# dictionary 생성
store_dict = {'가게 정보': []}
# 시작시간
start = time.time()
print('[크롤링 시작...]')

# 크롤링 (페이지 리스트 만큼)
for btn in range(len(next_btn))[1:]:  # next_btn[0] = 이전 페이지 버튼 무시 -> [1]부터 시작
    store_list = driver.find_elements(By.CSS_SELECTOR, 'li.VLTHu')
    
    names = driver.find_elements(By.CSS_SELECTOR, '.YwYLL')  # (3) 장소명
for data in range(len(store_list)): 

    sleep(2)
    try:
        # 도로명 초기화
        road_address = ''

        # 가게명
        store_name = names[data].text
    
        # 주소 버튼 누르기
        address_buttons = driver.find_elements(By.CSS_SELECTOR, '.lWwyx > a')
        address_buttons[data].click()

        sleep(2)

        # 주소 눌렀을 때 도로명, 지번 나오는 div
        addr = driver.find_elements(By.CSS_SELECTOR, '.AbTyi> div')
    
        sleep(2)
        
        # 도로명
        road = addr[0].text 
        road_address = road[3:-2]
        words = road_address.split()
        address = ' '.join(words[:4])
        sleep(2)
        
        if driver.find_elements(By.CSS_SELECTOR, '.TTfa9 > a'):
            img = driver.find_elements(By.CSS_SELECTOR, '.TTfa9 > a')
            img[data].click()
        
            sleep(2)
            
            imgUrl = driver.find_elements(By.CSS_SELECTOR, '.place_thumb > img')
            imgUrl = imgUrl[data].get_attribute("src")
        else:
            imgUrl =""

    except Exception as e:
        print(e)
        imgUrl=""
    print({'id':data, 'title': store_name, 'address':address, 'lat':geocoding(address)[0],'lng':geocoding(address)[1], 'imgUrl': imgUrl})
# 다음 페이지 버튼 누를 수 없으면 종료
    if not next_btn or not next_btn[-1].is_enabled():
        break

    if names[-1]:  # 마지막 가게일 경우 다음버튼 클릭
        next_btn[-1].click()
        sleep(2)

    else:
        print('페이지 인식 못함')
        break

print('[데이터 수집 완료]\n소요 시간 :', time.time() - start)
driver.quit()  # 작업이 끝나면 창을 닫음

[크롤링 시작...]
{'id': 0, 'title': '경주십원빵', 'address': '경북 경주시 포석로1092번길 70', 'lat': '35.8348555378555', 'lng': '129.215442410085', 'imgUrl': 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=f160_160&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20221011_256%2F1665499848075PY9c8_JPEG%2F9AB5F927-565B-4C2C-8B06-A7734DABA581.jpeg'}
{'id': 1, 'title': '경주십원빵', 'address': '서울 마포구 동교로 230', 'lat': '37.5604280520518', 'lng': '126.924465568225', 'imgUrl': 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=f160_160&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220603_257%2F1654218911906IbreN_JPEG%2FKakaoTalk_20220603_095946781_09.jpg'}
{'id': 2, 'title': '경주십원빵', 'address': '인천 중구 은하수로 14', 'lat': '37.4920818658842', 'lng': '126.58076999599', 'imgUrl': 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=f160_160&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20240630_137%2F1719724964694vIU82_JPEG%2F1000002317.jpg'}
{'id': 3, 'title': '경주십원빵', 'address': '충남 

In [17]:
from geopy.geocoders import Nominatim
geo_local = Nominatim(user_agent='South Korea')

import requests
# 위도, 경도 반환 함수
def geocoding(address):
    try:
        url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
        headers = {"Authorization": "KakaoAK b661426628e6bb134e2766ad9592dd33"}
        api_json = json.loads(str(requests.get(url,headers=headers).text))
        address = api_json['documents'][0]['address']
        x_y = [str(address['y']), str(address['x'])]
        return x_y
    except:
        x_y = ['', '']
        return x_y
    
# 네이버 지도 검색 페이지 열기
driver.get("https://map.naver.com/v5/search")
key_word = '경주 십원빵'  # 검색어

# css 찾을때 까지 10초대기
def time_wait(num, code):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait

# frame 변경 메소드
def switch_frame(frame):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경

# 페이지 다운
def page_down(num):
    body = driver.find_element(By.CSS_SELECTOR, 'body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)

# css를 찾을때 까지 10초 대기
time_wait(10, 'div.input_box > input.input_search')

# 검색창 찾기
search = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
search.send_keys(key_word)  
search.send_keys(Keys.ENTER) 

sleep(5)

# frame 변경
time_wait(10, 'iframe#searchIframe')
switch_frame('searchIframe')

sleep(5)

page_down(40)
sleep(2)

# 가게 리스트
store_list = driver.find_elements(By.CSS_SELECTOR, 'li.VLTHu')
# 페이지 리스트
next_btn = driver.find_elements(By.CSS_SELECTOR, '.zRM9F> a')

# dictionary 생성
store_dict = []
# 시작시간
start = time.time()
print('[크롤링 시작...]')

# 크롤링 (페이지 리스트 만큼)
for btn in range(len(next_btn))[1:]:  # next_btn[0] = 이전 페이지 버튼 무시 -> [1]부터 시작
    store_list = driver.find_elements(By.CSS_SELECTOR, 'li.VLTHu')
    
    names = driver.find_elements(By.CSS_SELECTOR, '.YwYLL')  # (3) 장소명
for data in range(len(store_list)): 

    sleep(2)
    try:
        # 도로명 초기화
        road_address = ''

        # 가게명
        store_name = names[data].text
    
        # 주소 버튼 누르기
        address_buttons = driver.find_elements(By.CSS_SELECTOR, '.lWwyx > a')
        address_buttons[data].click()

        sleep(2)

        # 주소 눌렀을 때 도로명, 지번 나오는 div
        addr = driver.find_elements(By.CSS_SELECTOR, '.AbTyi> div')
    
        sleep(2)
        
        # 도로명
        road = addr[0].text 
        road_address = road[3:-2]
        words = road_address.split()
        address = ' '.join(words[:4])
        sleep(2)
        
        if driver.find_elements(By.CSS_SELECTOR, '.TTfa9 > a'):
            img = driver.find_elements(By.CSS_SELECTOR, '.TTfa9 > a')
            img[data].click()
        
            sleep(2)
            
            imgUrl = driver.find_elements(By.CSS_SELECTOR, '.place_thumb > img')
            imgUrl = imgUrl[data].get_attribute("src")
        else:
            imgUrl =""

    except Exception as e:
        print(e)
        imgUrl=""
    print({'id':data, 'title': store_name, 'address':address, 'lat':geocoding(address)[0],'lng':geocoding(address)[1], 'imgUrl': imgUrl})
    dict_temp = {
                'id' : data,
                'title': store_name,
                'address': address,
                'lat' : geocoding(road_address)[0],
                'lng' : geocoding(road_address)[1],
                'imgUrl' : imgUrl
            }

    store_dict.append(dict_temp)
# 다음 페이지 버튼 누를 수 없으면 종료
    if not next_btn or not next_btn[-1].is_enabled():
        break

    if names[-1]:  # 마지막 가게일 경우 다음버튼 클릭
        next_btn[-1].click()
        sleep(2)

    else:
        print('페이지 인식 못함')
        break

print('[데이터 수집 완료]\n소요 시간 :', time.time() - start)
driver.quit()  # 작업이 끝나면 창을 닫음

with open('data/geongjyBbang_data.json', 'w', encoding='utf-8') as f:
   json.dump(store_dict, f, indent=4, ensure_ascii=False)

[크롤링 시작...]
{'id': 0, 'title': '경주십원빵', 'address': '경북 경주시 포석로1092번길 70', 'lat': '35.8348555378555', 'lng': '129.215442410085', 'imgUrl': 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=f160_160&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20221011_256%2F1665499848075PY9c8_JPEG%2F9AB5F927-565B-4C2C-8B06-A7734DABA581.jpeg'}
{'id': 1, 'title': '경주십원빵', 'address': '서울 마포구 동교로 230', 'lat': '37.5604280520518', 'lng': '126.924465568225', 'imgUrl': 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=f160_160&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220603_257%2F1654218911906IbreN_JPEG%2FKakaoTalk_20220603_095946781_09.jpg'}
{'id': 2, 'title': '경주십원빵', 'address': '인천 중구 은하수로 14', 'lat': '37.4920818658842', 'lng': '126.58076999599', 'imgUrl': 'https://search.pstatic.net/common/?autoRotate=true&quality=95&type=f160_160&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20240630_137%2F1719724964694vIU82_JPEG%2F1000002317.jpg'}
{'id': 3, 'title': '경주십원빵', 'address': '충남 